In [95]:
import os
os.chdir('../')

In [94]:
%load_ext autoreload
%autoreload 2

In [97]:
from search import *
from youtube_transcript_api import YouTubeTranscriptApi
import requests
from bs4 import BeautifulSoup
from youtubesearchpython import VideosSearch

In [10]:
api_key = ''

{'status': 'success', 'sharing_url': 'https://300.ya.ru/7C4mjYIu'}

In [24]:
response = response.json()
if response['status'] == 'success':
    url = response['sharing_url']
    resp = requests.get(url)
resp.status_code

200

In [31]:
from bs4 import BeautifulSoup
root = BeautifulSoup(resp.content.decode('utf-8'))
print(str(root.find(attrs={'class':'summary-text'})))

<div class="summary-text svelte-tiidzr"><h1 class="title svelte-1xfo2l">Как правильно анализировать конкурентов – Учебка ЛидМашины </h1> <ul class="theses svelte-11a0vor"><li class="thesis svelte-11a0vor thesis-link"><a class="svelte-11a0vor" href="https://leadmachine.ru/2019/03/22/kak-pravilno-analizirovat-konkurentov/#:~:text=%D0%92%D0%BE%D1%82%20%D0%BA%D0%BE%D0%BC%D0%BF%D0%B0%D0%BD%D0%B8%D0%B8%2D%D0%BA%D0%BE%D0%BD%D0%BA%D1%83%D1%80%D0%B5%D0%BD%D1%82%D1%8B%20%D0%BD%D0%B0%D1%88%D0%B5%D0%B3%D0%BE%20%D0%B8%D0%BD%D1%82%D0%B5%D1%80%D0%BD%D0%B5%D1%82%2D%D0%BC%D0%B0%D0%B3%D0%B0%D0%B7%D0%B8%D0%BD%D0%B0%20%D0%BB%D1%8E%D1%81%D1%82%D1%80%3A" target="_blank"> <pre class="thesis-content svelte-1g58soo"><p class="thesis-text svelte-1g58soo"><span class="text-wrapper"><span class="hidden-bullet svelte-1g58soo" data-svelte-h="svelte-nkm90c">• </span>Статья предлагает анализ конкурентов для интернет-магазинов люстр.</span><span class="icon-wrapper svelte-1g58soo"> <img alt="Ссылка" class="icon link-i

In [29]:
from youtubesearchpython import VideosSearch

videosSearch = VideosSearch('NoCopyrightSounds', limit = 1)

urls = list(map(lambda x: f'youtube.com/watch?v={x["id"]}', videosSearch.result()['result']))
print(urls)

['youtube.com/watch?v=oWW5TLrrbNo']


In [89]:
from youtubesearchpython import VideosSearch
from bs4 import BeautifulSoup
import requests

def find_youtube_video(query):
    videosSearch = VideosSearch(query, limit = 1)
    # urls = list(map(lambda x: f'https://youtube.com/watch?v={x["id"]}', videosSearch.result()['result']))
    try:
        return videosSearch.result()['result'][0]['id']
    except Exception as e:
        print(e)
    return None

def send_url_to_summarize(url, yandex_api_key):
    endpoint = 'https://300.ya.ru/api/sharing-url'
    response = requests.post(
        endpoint,
        json = {
          'article_url': url
        },
        headers = {'Authorization': f'OAuth {yandex_api_key}'}
    )
    response = response.json()
    if response['status'] != 'success':
        print(response['status'])
        print(response)
        return None
    return response['sharing_url']

def parse_300_ya_ru(sharing_url):
    resp = requests.get(sharing_url)
    root = BeautifulSoup(resp.content.decode('utf-8'))
    return root.find(attrs={'class':'summary-text'})


def summarize_video(query, yandex_api_key):
    url = find_youtube_video(query)
    print(url)
    # url = 'https://www.youtube.com/watch?v=iFfN116ATQg'
    # url = 'https://habr.com'
    sharing_url = send_url_to_summarize(url, yandex_api_key)
    if sharing_url is None:
        return None
    print(sharing_url)
    return parse_300_ya_ru(sharing_url)
    

In [76]:
res = summarize_video('новости', api_key)
res.text

https://youtube.com/watch?v=n84fl_HaXjs
error
{'status': 'error', 'message': 'not found'}


AttributeError: 'NoneType' object has no attribute 'text'

In [100]:
def get_video_transcription(query):
    video_id = find_youtube_video(query)
    print(f'https://youtube.com/watch?v={video_id}')
    transcribe = YouTubeTranscriptApi.get_transcript(video_id, languages=('ru',))
    total_len = 0
    i = 0
    lines = []
    while total_len < 5000 and i < len(transcribe):
        text = transcribe[i]['text']
        lines.append(text)
        total_len += len(text)
        i += 1

    transcribe = ' '.join(lines)
    return transcribe

In [102]:
search_api_key = ''
video = get_video_transcription('рынок газа')
summarization = ask_yagpt(make_prompt_by_html(video), search_api_key, 'yandexgpt')
summarization

https://youtube.com/watch?v=lggg2Fzry3k


'**Краткий отчёт по анализу общего рынка газа Евразийского экономического союза (ЕАЭС)**\n\n1. Решение о создании общего рынка газа ЕАЭС было принято главами государств-членов и закреплено в договоре ЕАЭС от 9 мая 2014 года.\n\n2. Основными целями формирования общего рынка газа являются повышение экономической эффективности использования газотранспортных систем, повышение энергетической безопасности государств-членов, а также надёжности, доступности и качества газоснабжения.\n\n3. Формирование общего рынка газа осуществляется в соответствии с основными принципами формирования общих энергетических рынков, закреплёнными в статье 79 договора ЕАЭС и основных принципах приложения 22.\n\n4. Газовые рынки государств-членов ЕАЭС имеют различный уровень развития и обеспеченности газом, что определяет цены на данный ресурс внутри стран и их участие в международной торговле.\n\n5. Целевая модель общего рынка газа предусматривает взаимную торговлю газом между участниками общего рынка всех государс